# Task-2: Wind Power Forecasting With Multiple Targets
Author: Karan Singh

In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
labels = pd.read_parquet(r'../Case_Interview_-_Multitarget_Power_Forecasting/Case Interview - Multitarget Power Forecasting/data/labels.parquet', engine='pyarrow')
weather = pd.read_parquet(r'../Case_Interview_-_Multitarget_Power_Forecasting/Case Interview - Multitarget Power Forecasting/data/weather_forecast.parquet', engine='pyarrow')

In [3]:
# Ensure that the labels are in 15 minute resolution
labels_index = pd.date_range(start=labels.index.min(), end=labels.index.max(), freq='15min')
labels.index = labels_index

In [4]:
# Add features -> part of day & Month
# Could temperature play a part? Is it cooler in the evenings/night?  
# Can the months represent seasons?    

conditions = [(0 <= labels.index.hour) & (labels.index.hour < 6), 
            (6 <= labels.index.hour) & (labels.index.hour < 12), 
            (12 <= labels.index.hour) & (labels.index.hour < 18), 
            (18 <= labels.index.hour) & (labels.index.hour < 24)]
choices = [0, 1, 2, 3]
labels['Day_part'] = np.select(condlist=conditions, choicelist=choices, default=np.nan)
labels['Month'] = labels.index.month
labels = labels[['Day_part', 'Month', 'power_1', 'power_2', 'power_3']]

In [5]:
labels.head()

,Day_part,Month,power_1,power_2,power_3
2019-01-01 00:00:00,0.0,1,48637.0,51637.0,54637.0
2019-01-01 00:15:00,0.0,1,52357.0,55357.0,58357.0
2019-01-01 00:30:00,0.0,1,54317.0,57317.0,60317.0
2019-01-01 00:45:00,0.0,1,54220.5,57220.5,60220.5
2019-01-01 01:00:00,0.0,1,51680.0,54680.0,57680.0


In [6]:
weather.head()

,Generation_Date,Forecast_Date,U,V,ws,Direction
1,2019-06-25,2019-06-25 01:00:00,1.861707,0.195582,1.871952,264.002783
2,2019-06-25,2019-06-25 02:00:00,1.257695,0.570511,1.381043,245.600179
3,2019-06-25,2019-06-25 03:00:00,1.883554,0.701067,2.009794,249.584514
4,2019-06-25,2019-06-25 04:00:00,2.664914,0.169917,2.670325,266.351713
5,2019-06-25,2019-06-25 05:00:00,2.855113,-0.255226,2.866498,275.108242


In [7]:
# Drop Generation_Date and make Forecast_Date the index 
weather = weather.drop('Generation_Date', axis=1).set_index('Forecast_Date')

In [8]:
weather.head()

,U,V,ws,Direction
Forecast_Date,,,,
2019-06-25 01:00:00,1.861707,0.195582,1.871952,264.002783
2019-06-25 02:00:00,1.257695,0.570511,1.381043,245.600179
2019-06-25 03:00:00,1.883554,0.701067,2.009794,249.584514
2019-06-25 04:00:00,2.664914,0.169917,2.670325,266.351713
2019-06-25 05:00:00,2.855113,-0.255226,2.866498,275.108242


In [9]:
# Merge the labels and weather data
data = pd.merge(left=labels, right=weather, left_index=True, right_index=True, how='left')

In [10]:
data.head()

,Day_part,Month,power_1,power_2,power_3,U,V,ws,Direction
2019-01-01 00:00:00,0.0,1,48637.0,51637.0,54637.0,NaN,NaN,NaN,NaN
2019-01-01 00:15:00,0.0,1,52357.0,55357.0,58357.0,NaN,NaN,NaN,NaN
2019-01-01 00:30:00,0.0,1,54317.0,57317.0,60317.0,NaN,NaN,NaN,NaN
2019-01-01 00:45:00,0.0,1,54220.5,57220.5,60220.5,NaN,NaN,NaN,NaN
2019-01-01 01:00:00,0.0,1,51680.0,54680.0,57680.0,NaN,NaN,NaN,NaN


In [11]:
# Keep values from the start date of weather 
data = data[np.where(data.index == weather.index.min())[0][0]:]

In [12]:
# Fill the hourly values forward to quarter hour 
data['U'] = data['U'].fillna(method='ffill')
data['V'] = data['V'].fillna(method='ffill')
data['ws'] = data['ws'].fillna(method='ffill')
data['Direction'] = data['Direction'].fillna(method='ffill')

In [13]:
# Rearrange so that power is the last column
data = data[['Day_part', 'Month', 'U', 'V', 'ws', 'Direction', 'power_1','power_2','power_3']]

In [14]:
data.head()

,Day_part,Month,U,V,ws,Direction,power_1,power_2,power_3
2019-06-25 01:00:00,0.0,6,1.861707,0.195582,1.871952,264.002783,4686.2,7686.2,10686.2
2019-06-25 01:15:00,0.0,6,1.861707,0.195582,1.871952,264.002783,4047.9,7047.9,10047.9
2019-06-25 01:30:00,0.0,6,1.861707,0.195582,1.871952,264.002783,3251.0,6251.0,9251.0
2019-06-25 01:45:00,0.0,6,1.861707,0.195582,1.871952,264.002783,2244.5,5244.5,8244.5
2019-06-25 02:00:00,0.0,6,1.257695,0.570511,1.381043,245.600179,2224.6,5224.6,8224.6


#### Use Random Forest Regression to impute missing power values

In [15]:
missing_powers_mask = (data['power_1'].isnull()) & (data['power_2'].isnull()) & (data['power_3'].isnull())   
data.loc[missing_powers_mask].shape[0]

341

In [16]:
powers_present_mask = (data['power_1'].notnull()) & (data['power_2'].notnull()) & (data['power_3'].notnull())
data[powers_present_mask]
rf_inputs = data[powers_present_mask].drop(['power_1', 'power_2', 'power_3'], axis=1)
rf_labels = data[powers_present_mask][['power_1','power_2', 'power_3']]

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

In [18]:
X_train, X_test, y_train, y_test = train_test_split(rf_inputs, rf_labels, test_size=0.20)

In [19]:
regr = MultiOutputRegressor(RandomForestRegressor(n_estimators=50))
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

0.9695357860634299

In [20]:
# Get the values that are missing
pred_data = data.loc[missing_powers_mask].drop(['power_1', 'power_2', 'power_3'], axis=1)

In [21]:
predictions = regr.predict(pred_data)

In [22]:
for ind, pred in zip(data[missing_powers_mask].index, predictions):
    data.loc[ind,'power_1'] = pred[0]
    data.loc[ind,'power_2'] = pred[1]
    data.loc[ind,'power_3'] = pred[2]

In [23]:
data.isna().sum()

Day_part     0
Month        0
U            0
V            0
ws           0
Direction    0
power_1      0
power_2      0
power_3      0
dtype: int64

### Model Building

In [24]:
np.set_printoptions(suppress=True)

In [25]:
# Date to split the training and testing
index_1Aug2020 = np.where(data.index == '2020-08-01 00:00:00')[0][0] 

training_data = data[:index_1Aug2020] 
testing_data = data[index_1Aug2020:]

In [26]:
# Ensure that training data starts at 00:15:00 and ends at 00:00:00
training_data = training_data.loc['2019-06-26 00:15:00':'2020-07-30 00:00:00']

In [27]:
training_data.head()

,Day_part,Month,U,V,ws,Direction,power_1,power_2,power_3
2019-06-26 00:15:00,0.0,6,2.202267,1.753324,2.814982,231.475115,7021.0,10021.0,13021.0
2019-06-26 00:30:00,0.0,6,2.202267,1.753324,2.814982,231.475115,6050.6,9050.6,12050.6
2019-06-26 00:45:00,0.0,6,2.202267,1.753324,2.814982,231.475115,7160.7,10160.7,13160.7
2019-06-26 01:00:00,0.0,6,1.593115,0.233521,1.610139,261.660883,6447.0,9447.0,12447.0
2019-06-26 01:15:00,0.0,6,1.593115,0.233521,1.610139,261.660883,5422.2,8422.2,11422.2


In [28]:
# Take all rows except the last 96 instances and all columns except the power columns
training_X = training_data.iloc[:-96,:6]

In [29]:
training_X.shape

(38304, 6)

In [30]:
# Split the training data into chunks of 96
X = np.array(np.split(training_X.values, training_X.shape[0]//96))
X.shape

(399, 96, 6)

In [31]:
# Split into train and test sets
X_train = X[:320]
X_test = X[320:]

In [32]:
# Only power columns
training_y = training_data.loc[:][['power_1', 'power_2', 'power_3']]
training_y.head()

,power_1,power_2,power_3
2019-06-26 00:15:00,7021.0,10021.0,13021.0
2019-06-26 00:30:00,6050.6,9050.6,12050.6
2019-06-26 00:45:00,7160.7,10160.7,13160.7
2019-06-26 01:00:00,6447.0,9447.0,12447.0
2019-06-26 01:15:00,5422.2,8422.2,11422.2


In [33]:
# Drop the first 96 instances of the power columns
training_y = training_y.iloc[96:]
training_y.shape

(38304, 3)

In [34]:
from sklearn.preprocessing import MinMaxScaler
sc_X = MinMaxScaler(feature_range=(0, 1))

In [35]:
def scale_3d(array, scaler)-> np.ndarray:
    days, instances, features = array.shape
    array_2d = np.reshape(array, newshape=(days, instances* features))
    scaled_2d = scaler(array_2d)
    scaled_3d = np.reshape(scaled_2d, newshape=(days, instances, features))
    return scaled_3d

In [36]:
X_train = scale_3d(X_train, sc_X.fit_transform)
X_test = scale_3d(X_test, sc_X.transform)

In [37]:
# Flatten X
n_input = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape((X_train.shape[0], n_input))
X_train.shape

(320, 576)

In [38]:
n_input

576

In [39]:
# Assign power_1 to y_1 and split it into 96 daily values 
y_1 = training_y['power_1'].values
y_1 = np.array(np.split(y_1, y_1.shape[0]/96))
y_1.shape

(399, 96)

In [40]:
# Assign power_2 to y_2 and split it into 96 daily values 
y_2 = training_y['power_2'].values
y_2 = np.array(np.split(y_2, y_2.shape[0]/96))
y_2.shape

(399, 96)

In [41]:
# Assign power_3 to y_3 and split it into 96 daily values 
y_3 = training_y['power_3'].values
y_3 = np.array(np.split(y_3, y_3.shape[0]/96))
y_3.shape

(399, 96)

In [42]:
# Split into train and test sets
y_train_1 = y_1[:320]
y_test_1 = y_1[320:]

y_train_2 = y_2[:320]
y_test_2 = y_2[320:]

y_train_3 = y_3[:320]
y_test_3 = y_3[320:]

In [43]:
import tensorflow as tf
import keras
from keras.layers import Dense, Layer

In [44]:
# Setup the switch variable
switch = tf.Variable(1)

In [45]:
class FiLMLayer(Layer):
    def __init__(self, switch):
      super(FiLMLayer, self).__init__()
      self.switch = switch
      self.gamma = tf.Variable(initial_value=tf.random.uniform(shape=(3,200), minval=0, maxval=1, dtype=tf.dtypes.float32), trainable=True) 
      self.beta = tf.Variable(initial_value=tf.random.uniform(shape=(3,200), minval=0, maxval=1, dtype=tf.dtypes.float32), trainable=True)

    def call(self, input):
      if self.switch == 1:
        return tf.multiply(self.gamma[0], input) + self.beta[0]
      elif self.switch == 2:
        return tf.multiply(self.gamma[1], input) + self.beta[1]
      else: 
        return tf.multiply(self.gamma[2], input) + self.beta[2]

In [46]:
# Create a network
inputLayer = keras.Input(shape=(n_input))
dense_layer2= Dense(200, activation='relu', name="Dense1")(inputLayer)
film_layer = FiLMLayer(switch)(dense_layer2)
dense_layer2 = Dense(200, activation='relu', name="Dense2")(film_layer)
outputLayer = Dense(96)(dense_layer2) 

In [47]:
model = keras.Model(inputs=inputLayer, outputs=outputLayer)

In [48]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 576)]             0         
                                                                 
 Dense1 (Dense)              (None, 200)               115400    
                                                                 
 fi_lm_layer (FiLMLayer)     (None, 200)               1201      
                                                                 
 Dense2 (Dense)              (None, 200)               40200     
                                                                 
 dense (Dense)               (None, 96)                19296     
                                                                 
Total params: 176,097
Trainable params: 176,097
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.compile(loss='mse', optimizer='adam')

In [50]:
# switch variable set to 1 previously
model.fit(X_train, y_train_1,  epochs=5, batch_size=1, verbose=1)

Epoch 1/5
320/320 [==============================] - 5s 5ms/step - loss: 893127552.0000
Epoch 2/5
320/320 [==============================] - 2s 5ms/step - loss: 458929152.0000
Epoch 3/5
320/320 [==============================] - 2s 5ms/step - loss: 437485152.0000
Epoch 4/5
320/320 [==============================] - 2s 5ms/step - loss: 419380256.0000
Epoch 5/5
320/320 [==============================] - 2s 5ms/step - loss: 405228960.0000


In [51]:
# Set switch to 2 and fit model
switch.assign(2)
model.fit(X_train, y_train_2,  epochs=5, batch_size=1, verbose=1)

Epoch 1/5
320/320 [==============================] - 2s 5ms/step - loss: 388578368.0000
Epoch 2/5
320/320 [==============================] - 2s 5ms/step - loss: 359713504.0000
Epoch 3/5
320/320 [==============================] - 2s 5ms/step - loss: 338873952.0000
Epoch 4/5
320/320 [==============================] - 2s 5ms/step - loss: 317741760.0000
Epoch 5/5
320/320 [==============================] - 2s 7ms/step - loss: 299890848.0000


In [52]:
# Set switch to 3 and fit model
switch.assign(3)
model.fit(X_train, y_train_3, epochs=5, batch_size=1, verbose=1)

Epoch 1/5
320/320 [==============================] - 2s 5ms/step - loss: 301728096.0000
Epoch 2/5
320/320 [==============================] - 2s 5ms/step - loss: 277100768.0000
Epoch 3/5
320/320 [==============================] - 2s 5ms/step - loss: 265148256.0000
Epoch 4/5
320/320 [==============================] - 2s 5ms/step - loss: 255550384.0000
Epoch 5/5
320/320 [==============================] - 2s 6ms/step - loss: 250683296.0000


In [53]:
# Flatten test
n_input2 = X_test.shape[1] * X_test.shape[2]  
X_test = X_test.reshape(X_test.shape[0], n_input2)
X_test.shape

(79, 576)

In [54]:
def nmae(y_true, y_pred):
    ''' Normalized Mean Absolute Error '''
    return 100*((np.sum(np.abs(y_true - y_pred))/y_true.size)/106400)

In [55]:
# switch variable has been set to 3 previously
# test with power_3 labels
yhat3 = model.predict(X_test)
nmae_test3 = nmae(y_test_3, yhat3)
nmae_test3

3/3 [==============================] - 1s 6ms/step


13.72860019692104

In [56]:
# Set switch to 2
# test with power_2 labels
switch.assign(2)
yhat2 = model.predict(X_test)
nmae_test2 = nmae(y_test_2, yhat2)
nmae_test2

3/3 [==============================] - 0s 5ms/step


17.015071528974357

In [57]:
# Set switch to 1
# test with power_1 labels
switch.assign(1)
yhat1 = model.predict(X_test)
nmae_test2 = nmae(y_test_1, yhat1)
nmae_test2

3/3 [==============================] - 0s 4ms/step


20.167220733933128

## Making predictions on test data

In [58]:
testing_data.shape

(8737, 9)

In [59]:
# Delete the first instance at 00:00:00
testing_data_div96 = testing_data[1:]
testing_data_div96.shape

(8736, 9)

In [60]:
# Drop the last 96 instances for x & delete the power columns
test_x = testing_data_div96.iloc[:-96,:6]
# Drop the first 96 instances for y and delete the features
test_y = testing_data_div96.iloc[96:,6:]

In [61]:
test_x.shape

(8640, 6)

In [62]:
# Split the test data into chunks of 96
test_x_split = np.array(np.split(test_x.values, test_x.shape[0]//96))
test_x_split.shape

(90, 96, 6)

In [63]:
# Flatten X
n_input3 = test_x_split.shape[1] * test_x_split.shape[2]
test_x_unscaled = test_x_split.reshape(test_x_split.shape[0], n_input3)
test_x_unscaled.shape

(90, 576)

In [64]:
# Normalizing
text_x_scaled = sc_X.fit_transform(test_x_unscaled)
text_x_scaled.shape

(90, 576)

In [65]:
# Set switch to 1 and predict
switch.assign(1)
test_yhat1 = model.predict(text_x_scaled)

3/3 [==============================] - 0s 4ms/step


In [66]:
# Set switch to 2 and predict
switch.assign(2)
test_yhat2 = model.predict(text_x_scaled)

3/3 [==============================] - 0s 6ms/step


In [67]:
# Set switch to 3 and predict
switch.assign(3)
test_yhat3 = model.predict(text_x_scaled)

3/3 [==============================] - 0s 5ms/step


In [68]:
# Get power_1 labels
# Split labels by days
# Calculate the NMAE for days and predictions
target1 = test_y['power_1'].values
target1_split = np.array(np.split(target1, 90))
nmae1 = np.array([nmae(target, yhat) for target, yhat in zip(target1_split, test_yhat1)])

In [69]:
# Get power_2 labels
# Split labels by days
# Calculate the NMAE for days and predictions
target2 = test_y['power_2'].values
target2_split = np.array(np.split(target2, 90))
nmae2 = np.array([nmae(target, yhat) for target, yhat in zip(target2_split, test_yhat2)])

In [70]:
# Get power_3 labels
# Split labels by days
# Calculate the NMAE for days and predictions
target3 = test_y['power_3'].values
target3_split = np.array(np.split(target3, 90))
nmae3 = np.array([nmae(target, yhat) for target, yhat in zip(target3_split, test_yhat3)])

In [71]:
# Create a matrix (n_targets, n_days) 
target_days_matrix = np.empty((3,90))
target_days_matrix[0] = nmae1
target_days_matrix[1] = nmae2
target_days_matrix[2] = nmae3

In [72]:
target_days_matrix.shape

(3, 90)